## Rolling mean

In [1]:
import pandas as pd
import polars as pl
import numpy as np
import sys
#%load_ext line_profiler

df = pd.DataFrame({'B': np.random.randint(0,100, 100_000_000) / 100})
df['B'] = df['B'].astype(float)
WINDOW = 500

vals = df['B'].values
results = {}

### Naive

In [ ]:
def rolling_mean_v0(l, k):
    res = [np.nan for i in l]
    for i, val in enumerate(l):
        if i == (k-1):
            res[i] = sum(l[:(i+1)]) / k
        elif i >= k:
            res[i] = sum(l[(i-(k-1)):(i+1)]) / k
    return res

In [ ]:
df['v0'] = rolling_mean_v0(vals, WINDOW)

In [ ]:
v = %timeit -o rolling_mean_v0(vals, WINDOW)
results['v0'] = {"name": "naive", "time": v.average}

### v1

In [ ]:
def rolling_mean_v1():
    return df['B'].rolling(WINDOW).mean()

In [ ]:
df['v1'] = rolling_mean_v1()

In [ ]:
v = %timeit -o rolling_mean_v1()
results['v1'] = {"name": "pandas", "time": v.average}

### v2

In [ ]:
def rolling_mean_v2(l, k):
    s = 0.0
    c = 0
    res = [np.nan for i in l]
    for i, val in enumerate(l):
        if c < (k-1):
            s += val
            c += 1
        else:
            s -= (l[i-k] if i-k >=0 else 0)
            s += val
            res[i] = s / k
    return res
#%lprun -f rolling_mean_v2 rolling_mean_v2(vals, 3)

In [ ]:
v = %timeit -o rolling_mean_v2(vals, WINDOW)
results['v2'] = {"name": "window", "time": v.average}

In [ ]:
df['v2'] = rolling_mean_v2(vals, WINDOW)

###  v3

In [ ]:
def rolling_mean_v3(l, k):
    s = sum(l[:k])
    res = [np.nan]*len(l)
    res[k-1] = s / k
    for i, val in enumerate(l[(k):]):    
        s -= l[i]
        s += val
        res[i+(k)] = s / k
    return res

In [ ]:
df['v3'] = rolling_mean_v3(vals, WINDOW)

In [ ]:
v = %timeit -o rolling_mean_v3(vals, WINDOW)
results['v3'] = {"name": "window_less_branching", "time": v.average}

### v4

In [3]:
%load_ext Cython

In [ ]:
%%cython -a

cimport cython
import numpy as np
cimport numpy as np

def rolling_mean_v4(l, k):
    s = sum(l[:k])
    res = [np.nan]*len(l)
    res[k-1] = s / k
    for i, val in enumerate(l[(k):]):    
        s -= l[i]
        s += val
        res[i+(k)] = s / k
    return res

In [ ]:
v = %timeit -o rolling_mean_v4(vals, WINDOW)
results['v4'] = {"name": "cython_v1", "time": v.average}

In [ ]:
df['v4'] = np.array(rolling_mean_v4(vals, WINDOW))

### v5

In [ ]:
%%cython -a

cimport cython
import numpy as np
cimport numpy as np

cdef float NAN 
NAN = float("NaN") 

@cython.boundscheck(False)
def rolling_mean_v5(np.float64_t[:] l, int k):
    cdef:
        int i = 0
        np.float64_t s
        np.float64_t val
        cdef np.float64_t[:] res = np.empty(len(l))
    s = sum(l[:k])
    res[:k] = np.nan
    res[k-1] = s / k
    for i, val in enumerate(l[(k):]):
        s += (val - l[i])
        res[i+(k)] = s / k
        i += 1
    return res

In [ ]:
v = %timeit -o rolling_mean_v5(vals, WINDOW)
results['v5'] = {"name": "cython_v2_added_cdef", "time": v.average}

In [ ]:
df['v5'] = np.array(rolling_mean_v5(vals, WINDOW))

### v6

In [ ]:
%%cython -a

cimport cython
import numpy as np
cimport numpy as np

cdef float NAN 
NAN = float("NaN") 

@cython.boundscheck(False)
@cython.cdivision(True)
def rolling_mean_v6(np.float64_t[:] l, int k):
    cdef:
        int i = 0
        int len_l = len(l[(k):])
        float k_f = k
        np.float64_t s = 0.0
        np.float64_t val
        cdef np.float64_t[:] res = np.empty(len_l+k)
    s = sum(l[:k])
    res[:k] = np.nan
    res[k-1] = s / k_f
    while i < len_l:
        s += (l[i+k] - l[i])
        res[i+k] = s / k_f
        i += 1
    return res

In [ ]:
v = %timeit -o rolling_mean_v6(vals, WINDOW)
results['v6'] = {"name": "cython_v2_more_def_less_safety", "time": v.average}

In [ ]:
df['v6'] = np.array(rolling_mean_v6(vals, WINDOW))

### V7

In [ ]:
polars_df = pl.DataFrame({"A": vals})

def rolling_mean_v7(window_size):
    return polars_df.with_columns(
    rolling_mean=pl.col("A").rolling_mean(window_size=window_size),
)

df['v7'] = rolling_mean_v7(WINDOW)['rolling_mean'].to_numpy()

In [ ]:
v = %timeit -o rolling_mean_v7(WINDOW)
results['v7'] = {"name": "polars", "time": v.average}

### V8

In [4]:
%%cython -a

cimport cython
import numpy as np
cimport numpy as np
from libc.stdlib cimport malloc, free

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
def rolling_mean_v8(np.ndarray[double, ndim=1] l, int k):
    cdef:
        int i, n = l.shape[0]
        double* l_ptr = <double*>l.data
        double* res_ptr
        double s = 0.0
        double k_inv = 1.0 / k
        np.ndarray[double, ndim=1] res = np.empty(n, dtype=np.float64)
    
    res_ptr = <double*>res.data
    
    for i in range(k-1):
        res_ptr[i] = np.nan
    
    for i in range(k):
        s += l_ptr[i]
    
    res_ptr[k-1] = s * k_inv
    
    for i in range(k, n):
        s += l_ptr[i] - l_ptr[i-k]
        res_ptr[i] = s * k_inv
    
    return res

In [5]:
v = %timeit -o rolling_mean_v8(vals, WINDOW)
results['v8'] = {"name": "cython_ptr", "time": v.average}

443 ms ± 7.84 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
df['v8'] = rolling_mean_v8(vals, WINDOW)

In [7]:
rolling_mean_v8(vals, WINDOW)

array([    nan,     nan,     nan, ..., 0.4867 , 0.48652, 0.48736],
      shape=(100000000,))

# rst_wnd version using pyo3, ndarray, numpy and maturin

## Safe

In [8]:
from rst_wnd import rolling_mean_v9_safe

v = %timeit -o rolling_mean_v9_safe(vals, WINDOW)
df['v9_safe'] = rolling_mean_v9_safe(vals, WINDOW)
results['v9_safe'] = {"name": "rst_wnd_safe", "time": v.average}

464 ms ± 10.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Raw Pointers

In [ ]:
from rst_wnd import rolling_mean_v9_raw_pointers
v = %timeit -o rolling_mean_v9_raw_pointers(vals, WINDOW)

df['v9_raw_pointers'] = rolling_mean_v9_raw_pointers(vals, WINDOW)
results['v9_raw_pointers'] = {"name": "rst_wnd_raw_pointers", "time": v.average}

## Portable SIMD

In [ ]:
from rst_wnd import rolling_mean_v9_portable_simd
v = %timeit -o rolling_mean_v9_portable_simd(vals, WINDOW)

df['v9_portable_simd'] = rolling_mean_v9_portable_simd(vals, WINDOW)
results['v9_portable_simd'] = {"name": "rst_wnd_portable_simd", "time": v.average}

## SIMD intrinsics

In [ ]:
from rst_wnd import rolling_mean_v9_simd_intr
v = %timeit -o rolling_mean_v9_simd_intr(vals, WINDOW)

df['v9_simd_intr'] = rolling_mean_v9_simd_intr(vals, WINDOW)
results['v9_simd_intr'] = {"name": "rst_wnd_simd_intr", "time": v.average}

# Results

In [ ]:
df

In [ ]:
res_df = pd.DataFrame(results).T

In [ ]:
res_df.time.astype(float)

In [ ]:
# Compute the pairwise ratio matrix
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
res_df.time = res_df.time.astype(float)
res_df['time'] = pd.to_numeric(res_df['time'], errors='coerce')

# Compute the ratio matrix so that cell (i, j) is: (time_j / time_i)
times = res_df['time'].values
ratio_matrix = times.reshape(1, -1) / times.reshape(-1, 1)

# Set logarithmic normalization for the heatmap
norm = mcolors.LogNorm(vmin=ratio_matrix.min(), vmax=ratio_matrix.max())

# Plot the heatmap with logarithmic color scaling
plt.figure(figsize=(10, 8))
sns.heatmap(ratio_matrix, annot=True, fmt=".2f", cmap="viridis", norm=norm,
            xticklabels=res_df['name'], yticklabels=res_df['name'])
plt.title("Pairwise Relative Performance Heatmap\n(Row method is x-times faster than Column method)")
plt.xlabel("Compared to")
plt.ylabel("Method (row)")
plt.show()

During dev: pyximport


To package: https://cython.readthedocs.io/en/latest/src/userguide/source_files_and_compilation.html

```python
import pyximport

pyximport.install(setup_args={"include_dirs": numpy.get_include()})
```